In [4]:
import pandas as pd
import numpy as np
import random
import time
import multiprocessing
from sklearn.utils import resample
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from lightgbm import LGBMClassifier
from xgboost import XGBClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.ensemble import VotingClassifier
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV
from sklearn.svm import SVC

from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier

import itertools as it

# Load Dataset

The datasets already removed the first 2 weeks' observations.

In [5]:
%%time
oot = pd.read_csv('oot_after_feature_selection.csv',index_col=0).reset_index().drop(columns='index')
oot.head()

Wall time: 333 ms


,address_dob_1_day_cross_count,address_homephone_lag14_count,address_lag14_count,address_lag1_count,address_lag1_lag7_avg,address_lag30_count,address_lag7_count,address_name_14_day_cross_count,address_name_30_day_cross_count,address_name_7_day_cross_count,...,ssn_dob_lag14_count,ssn_dob_lag30_count,ssn_firstname_lag30_count,ssn_lag30_count,ssn_name_dob_lag14_count,ssn_name_dob_lag30_count,zip5_homephone_lag14_count,zip5_homephone_lag7_count,date,fraud_label
0,1,1,1,1,7.0,1,1,1,1,1,...,1,1,1,1,1,1,1,1,2016-11-01,0
1,1,1,1,1,7.0,1,1,1,1,1,...,1,1,1,1,1,1,1,1,2016-11-01,0
2,1,2,2,1,7.0,2,1,1,1,1,...,2,2,2,2,2,2,2,1,2016-11-01,0
3,1,1,1,1,7.0,1,1,1,1,1,...,1,1,1,1,1,1,1,1,2016-11-01,0
4,1,1,1,1,7.0,1,1,1,1,1,...,1,1,1,1,1,1,1,1,2016-11-01,0


In [6]:
%%time
tt = pd.read_csv('tt_after_feature_selection.csv',index_col=0).reset_index().drop(columns='index')
tt.head()

Wall time: 1.22 s


,address_dob_1_day_cross_count,address_homephone_lag14_count,address_lag14_count,address_lag1_count,address_lag1_lag7_avg,address_lag30_count,address_lag7_count,address_name_14_day_cross_count,address_name_30_day_cross_count,address_name_7_day_cross_count,...,ssn_dob_lag14_count,ssn_dob_lag30_count,ssn_firstname_lag30_count,ssn_lag30_count,ssn_name_dob_lag14_count,ssn_name_dob_lag30_count,zip5_homephone_lag14_count,zip5_homephone_lag7_count,date,fraud_label
0,1,1,1,1,7.0,1,1,1,1,1,...,1,1,1,1,1,1,1,1,2016-01-15,0
1,1,1,1,1,7.0,1,1,1,1,1,...,1,1,1,1,1,1,1,1,2016-01-15,0
2,1,1,1,1,7.0,1,1,1,1,1,...,1,1,1,1,1,1,1,1,2016-01-15,0
3,1,1,1,1,7.0,1,1,1,1,1,...,1,1,1,1,1,1,1,1,2016-01-15,0
4,1,1,1,1,7.0,1,1,1,1,1,...,1,1,1,1,1,1,1,1,2016-01-15,0


# Data Preprocessing

## Z_scale except fraud_label, date


In [7]:
%%time
tt_var = tt.copy()
oot_var = oot.copy()

tt_var.drop(columns = ['fraud_label','date'],inplace=True)
oot_var.drop(columns = ['fraud_label','date'],inplace=True)

scaler = StandardScaler()

tt_z = scaler.fit_transform(tt_var)
oot_z = scaler.fit_transform(oot_var)

tt_z = pd.DataFrame(tt_z)
tt_z.columns = tt_var.columns

oot_z = pd.DataFrame(oot_z)
oot_z.columns = oot_var.columns

oot_z['fraud_label'] = oot['fraud_label']
tt_z['fraud_label'] = tt['fraud_label']

oot_z['date'] = oot['date']
tt_z['date'] = tt['date']

Wall time: 838 ms


In [8]:
tt_z

,address_dob_1_day_cross_count,address_homephone_lag14_count,address_lag14_count,address_lag1_count,address_lag1_lag7_avg,address_lag30_count,address_lag7_count,address_name_14_day_cross_count,address_name_30_day_cross_count,address_name_7_day_cross_count,...,ssn_dob_lag14_count,ssn_dob_lag30_count,ssn_firstname_lag30_count,ssn_lag30_count,ssn_name_dob_lag14_count,ssn_name_dob_lag30_count,zip5_homephone_lag14_count,zip5_homephone_lag7_count,fraud_label,date
0,-0.049234,-0.071459,-0.089492,-0.051839,0.104045,-0.119574,-0.073331,-0.067958,-0.078407,-0.061523,...,-0.069033,-0.094869,-0.100198,-0.101202,-0.068884,-0.094555,-0.071995,-0.058592,0,2016-01-15
1,-0.049234,-0.071459,-0.089492,-0.051839,0.104045,-0.119574,-0.073331,-0.067958,-0.078407,-0.061523,...,-0.069033,-0.094869,-0.100198,-0.101202,-0.068884,-0.094555,-0.071995,-0.058592,0,2016-01-15
2,-0.049234,-0.071459,-0.089492,-0.051839,0.104045,-0.119574,-0.073331,-0.067958,-0.078407,-0.061523,...,-0.069033,-0.094869,-0.100198,-0.101202,-0.068884,-0.094555,-0.071995,-0.058592,0,2016-01-15
3,-0.049234,-0.071459,-0.089492,-0.051839,0.104045,-0.119574,-0.073331,-0.067958,-0.078407,-0.061523,...,-0.069033,-0.094869,-0.100198,-0.101202,-0.068884,-0.094555,-0.071995,-0.058592,0,2016-01-15
4,-0.049234,-0.071459,-0.089492,-0.051839,0.104045,-0.119574,-0.073331,-0.067958,-0.078407,-0.061523,...,-0.069033,-0.094869,-0.100198,-0.101202,-0.068884,-0.094555,-0.071995,-0.058592,0,2016-01-15
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
794991,-0.049234,-0.071459,-0.089492,-0.051839,0.104045,-0.119574,-0.073331,-0.067958,-0.078407,-0.061523,...,-0.069033,-0.094869,-0.100198,-0.101202,-0.068884,-0.094555,-0.071995,-0.058592,0,2016-10-31
794992,-0.049234,-0.071459,-0.089492,-0.051839,0.104045,-0.119574,-0.073331,-0.067958,-0.078407,-0.061523,...,-0.069033,-0.094869,-0.100198,-0.101202,-0.068884,-0.094555,-0.071995,-0.058592,0,2016-10-31
794993,-0.049234,-0.071459,-0.089492,-0.051839,0.104045,-0.119574,-0.073331,-0.067958,-0.078407,-0.061523,...,-0.069033,-0.094869,-0.100198,-0.101202,-0.068884,-0.094555,-0.071995,-0.058592,0,2016-10-31
794994,-0.049234,-0.071459,-0.089492,-0.051839,0.104045,-0.119574,-0.073331,-0.067958,-0.078407,-0.061523,...,-0.069033,-0.094869,-0.100198,-0.101202,-0.068884,-0.094555,-0.071995,-0.058592,0,2016-10-31


## Split Train Test

In [9]:
tt_z.head()

,address_dob_1_day_cross_count,address_homephone_lag14_count,address_lag14_count,address_lag1_count,address_lag1_lag7_avg,address_lag30_count,address_lag7_count,address_name_14_day_cross_count,address_name_30_day_cross_count,address_name_7_day_cross_count,...,ssn_dob_lag14_count,ssn_dob_lag30_count,ssn_firstname_lag30_count,ssn_lag30_count,ssn_name_dob_lag14_count,ssn_name_dob_lag30_count,zip5_homephone_lag14_count,zip5_homephone_lag7_count,fraud_label,date
0,-0.049234,-0.071459,-0.089492,-0.051839,0.104045,-0.119574,-0.073331,-0.067958,-0.078407,-0.061523,...,-0.069033,-0.094869,-0.100198,-0.101202,-0.068884,-0.094555,-0.071995,-0.058592,0,2016-01-15
1,-0.049234,-0.071459,-0.089492,-0.051839,0.104045,-0.119574,-0.073331,-0.067958,-0.078407,-0.061523,...,-0.069033,-0.094869,-0.100198,-0.101202,-0.068884,-0.094555,-0.071995,-0.058592,0,2016-01-15
2,-0.049234,-0.071459,-0.089492,-0.051839,0.104045,-0.119574,-0.073331,-0.067958,-0.078407,-0.061523,...,-0.069033,-0.094869,-0.100198,-0.101202,-0.068884,-0.094555,-0.071995,-0.058592,0,2016-01-15
3,-0.049234,-0.071459,-0.089492,-0.051839,0.104045,-0.119574,-0.073331,-0.067958,-0.078407,-0.061523,...,-0.069033,-0.094869,-0.100198,-0.101202,-0.068884,-0.094555,-0.071995,-0.058592,0,2016-01-15
4,-0.049234,-0.071459,-0.089492,-0.051839,0.104045,-0.119574,-0.073331,-0.067958,-0.078407,-0.061523,...,-0.069033,-0.094869,-0.100198,-0.101202,-0.068884,-0.094555,-0.071995,-0.058592,0,2016-01-15


In [10]:
X = tt_z.iloc[:,:-2].values
y = tt_z.iloc[:,-2].values
X_oot = oot_z.iloc[:,:-2].values
y_oot = oot_z.iloc[:,-2].values

In [11]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.25, random_state = 2021, stratify = y)

In [12]:
print('X_train:',X_train.shape)
print('X_test:',X_test.shape)
print('X_oot:',X_oot.shape)

X_train: (596247, 30)
X_test: (198749, 30)
X_oot: (166493, 30)


In [13]:
X_train

array([[-0.04923381, -0.07145897, -0.08949214, ..., -0.09455508,
        -0.07199507, -0.05859199],
       [-0.04923381, -0.07145897, -0.08949214, ..., -0.09455508,
        -0.07199507, -0.05859199],
       [-0.04923381, -0.07145897, -0.08949214, ..., -0.09455508,
        -0.07199507, -0.05859199],
       ...,
       [-0.04923381, -0.07145897, -0.08949214, ..., -0.09455508,
        -0.07199507, -0.05859199],
       [-0.04923381, -0.07145897, -0.08949214, ..., -0.09455508,
        -0.07199507, -0.05859199],
       [-0.04923381, -0.07145897, -0.08949214, ..., -0.09455508,
        -0.07199507, -0.05859199]])

## Throw away records that score either very high or very low

In [14]:
X_train_copy = X_train.copy()
arr = (X_train_copy>6) | (X_train_copy<-6) 
arr

array([[False, False, False, ..., False, False, False],
       [False, False, False, ..., False, False, False],
       [False, False, False, ..., False, False, False],
       ...,
       [False, False, False, ..., False, False, False],
       [False, False, False, ..., False, False, False],
       [False, False, False, ..., False, False, False]])

In [15]:
%%time
l = []
for i in range(len(arr)):
    if sum(arr[i])>0:
        l.append(i)

Wall time: 26.6 s


In [16]:
# of observations to throw away
len(l)

9668

In [17]:
len(X_train_copy)

596247

In [18]:
X_train_capped6 = np.delete(X_train_copy,l,0)
((X_train_capped6>6) | (X_train_capped6<-6)).any()

False

In [19]:
# of observations after capping
len(X_train_capped6)

586579

In [20]:
%%time
X_train_capped6 = pd.DataFrame(X_train_capped6)
X_train_capped6.columns = tt_z.columns[:-2]
X_train_capped6

Wall time: 0 ns


,address_dob_1_day_cross_count,address_homephone_lag14_count,address_lag14_count,address_lag1_count,address_lag1_lag7_avg,address_lag30_count,address_lag7_count,address_name_14_day_cross_count,address_name_30_day_cross_count,address_name_7_day_cross_count,...,name_dob_lag14_count,name_dob_lag30_count,ssn_dob_lag14_count,ssn_dob_lag30_count,ssn_firstname_lag30_count,ssn_lag30_count,ssn_name_dob_lag14_count,ssn_name_dob_lag30_count,zip5_homephone_lag14_count,zip5_homephone_lag7_count
0,-0.049234,-0.071459,-0.089492,-0.051839,0.104045,-0.119574,-0.073331,-0.067958,-0.078407,-0.061523,...,-0.069225,-0.09515,-0.069033,-0.094869,-0.100198,-0.101202,-0.068884,-0.094555,-0.071995,-0.058592
1,-0.049234,-0.071459,-0.089492,-0.051839,0.104045,-0.119574,-0.073331,-0.067958,-0.078407,-0.061523,...,-0.069225,-0.09515,-0.069033,-0.094869,-0.100198,-0.101202,-0.068884,-0.094555,-0.071995,-0.058592
2,-0.049234,-0.071459,-0.089492,-0.051839,0.104045,-0.119574,-0.073331,-0.067958,-0.078407,-0.061523,...,-0.069225,-0.09515,-0.069033,-0.094869,-0.100198,-0.101202,-0.068884,-0.094555,-0.071995,-0.058592
3,-0.049234,-0.071459,-0.089492,-0.051839,0.104045,-0.119574,-0.073331,-0.067958,-0.078407,-0.061523,...,-0.069225,-0.09515,-0.069033,-0.094869,-0.100198,-0.101202,-0.068884,-0.094555,-0.071995,-0.058592
4,-0.049234,-0.071459,-0.089492,-0.051839,0.104045,-0.119574,-0.073331,-0.067958,-0.078407,-0.061523,...,-0.069225,-0.09515,-0.069033,-0.094869,-0.100198,-0.101202,-0.068884,-0.094555,-0.071995,-0.058592
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
586574,-0.049234,-0.071459,-0.089492,-0.051839,0.104045,-0.119574,-0.073331,-0.067958,-0.078407,-0.061523,...,-0.069225,-0.09515,-0.069033,-0.094869,-0.100198,-0.101202,-0.068884,-0.094555,-0.071995,-0.058592
586575,-0.049234,-0.071459,-0.089492,-0.051839,0.104045,-0.119574,-0.073331,-0.067958,-0.078407,-0.061523,...,-0.069225,-0.09515,-0.069033,-0.094869,-0.100198,-0.101202,-0.068884,-0.094555,-0.071995,-0.058592
586576,-0.049234,-0.071459,-0.089492,-0.051839,0.104045,-0.119574,-0.073331,-0.067958,-0.078407,-0.061523,...,-0.069225,-0.09515,-0.069033,-0.094869,-0.100198,-0.101202,-0.068884,-0.094555,-0.071995,-0.058592
586577,-0.049234,-0.071459,-0.089492,-0.051839,0.104045,-0.119574,-0.073331,-0.067958,-0.078407,-0.061523,...,-0.069225,-0.09515,-0.069033,-0.094869,-0.100198,-0.101202,-0.068884,-0.094555,-0.071995,-0.058592


In [21]:
X_train = pd.DataFrame(X_train)
X_train.columns = tt_z.columns[:-2]
y_train = pd.DataFrame(y_train)

X_test = pd.DataFrame(X_test)
X_test.columns = tt_z.columns[:-2]
y_test = pd.DataFrame(y_test)

X_oot = pd.DataFrame(X_oot)
X_oot.columns = oot.columns[:-2]
y_oot = pd.DataFrame(y_oot)

In [22]:
y_train.columns = ['fraud_label']
y_test.columns = ['fraud_label']
y_oot.columns = ['fraud_label']
y_train =y_train['fraud_label']
y_test = y_test['fraud_label']
y_oot = y_oot['fraud_label']

In [23]:
X = pd.DataFrame(X)
X.columns = tt_z.columns[:-2]
y = pd.DataFrame(y)
y.columns = ['fraud_label']
y = y['fraud_label']

# Modeling

## Define functions

### FDR @3%

In [24]:
def fdr_score(y_pred_prob, y_obs, alpha=0.03):
    '''
    Calculate FDR@3%
    '''
    n_rows = int(round(len(y_obs)*alpha))
    numBads = sum(y_obs)
    df = pd.DataFrame([y_pred_prob,y_obs]).T
    df.columns = ['pred_prob','label']
    df = df.sort_values('pred_prob',ascending=False)
    return df.head(n_rows)['label'].sum()/numBads*100

### Resample

In [25]:
def trn_resample(X, y, good_bad_ratio=10):
    '''
    Undersample the good records in the training data to match the specified good_bad_ratio
    '''
    df = pd.concat([X, y], axis=1)
    goods = df.loc[df.fraud_label==0]
    bads = df.loc[df.fraud_label==1]
    n_bads = bads.shape[0]
    goods_undersampled = resample(goods,
                  replace=False,
                  n_samples=n_bads*good_bad_ratio)
    df_undersampled = pd.concat([goods_undersampled, bads])
    X_undersampled = df_undersampled.iloc[:,:-1].values
    y_undersampled = df_undersampled.iloc[:,-1].values
    return X_undersampled, y_undersampled

### Fit Average FDR

In [26]:
def fit_avg_fdr(model, k=5):
    '''
    Refitting model multiple times to get averaged FDR on training/testing/oot data
    '''
    fdr_trainList = []
    fdr_testList = []
    fdr_ootList = []
    accuracyList = []
    
    for i in range(k):
        X_train_undersampled, y_train_undersampled = trn_resample(X_train, y_train)
        model.fit(X_train_undersampled, y_train_undersampled)

        y_train_pred_prob = model.predict_proba(X_train)[:,1]
        y_test_pred_prob = model.predict_proba(X_test)[:,1]
        y_oot_pred_prob = model.predict_proba(X_oot)[:,1]

        fdr_trainList.append(fdr_score(y_train_pred_prob,y_train))
        fdr_testList.append(fdr_score(y_test_pred_prob,y_test))
        fdr_ootList.append(fdr_score(y_oot_pred_prob,y_oot))
        accuracyList.append(model.score(X_train_undersampled, y_train_undersampled))
    
    score_summary = np.array([fdr_trainList, fdr_testList, fdr_ootList, accuracyList]).T
    avg_scores = score_summary.mean(0)
    
    return score_summary, avg_scores

### Cross Validation FDR

In [27]:
def cross_val_fdr(model, X, y, cv=5, verbose=True):
    '''
    Performs k-fold cross validation and calculates the FDR on the validation set
    '''
    print('Start cross_val_fdr...')
    arrX = X.values
    arry = y.values
    n = len(arry)
    
    # Calculate fold sizes
    fold_size = int(n/cv)
    fold_sizes = [fold_size]*cv
    remainder = n - fold_size*cv
    for i in range(remainder):
        fold_sizes[i]+=1
        
    # Split folds
    folds_X = []
    folds_y = []
    for fs in fold_sizes:
        index = random.choices(range(len(arrX)), k=fs)
        folds_X.append(arrX[index,:])
        folds_y.append(arry[index])
        arrX = np.delete(arrX, index, 0)
        arry = np.delete(arry, index, 0)
        
    # Fit model
    FDRs = []
    new = True
    for i in range(cv):
        X_tst = folds_X[i]
        y_tst = folds_y[i]

        # Combine folds
        for j in [j for j in range(cv) if j!=i]:
            if new:
                X_trn = folds_X[j]
                y_trn = folds_y[j]
                new = False
            else:
                X_trn = np.concatenate((X_trn, folds_X[j]), axis=0)
                y_trn = np.concatenate((y_trn, folds_y[j]), axis=0)

        #print("Train:", len(X_trn))
        #print("Train fraud", sum(y_trn))
        #print("Test:", len(X_tst))
        #print("Test fraud", sum(y_tst))

        # Resample training set
        X_trn_df = pd.DataFrame(X_trn)
        X_trn_df.columns = X.columns
        y_trn_df = pd.DataFrame(y_trn)
        y_trn_df.columns = [y.name]
        X_trn_downsampled, y_trn_downsampled = trn_resample(X_trn_df, y_trn_df)

        #print("Train resampled:", len(X_trn_downsampled))
        #print("Train resampled fraud:", sum(y_trn_downsampled))

        # Calculate FDR
        model.fit(X_trn_downsampled, y_trn_downsampled)
        y_pred_prob = model.predict_proba(X_tst)[:,1]
        fdr = fdr_score(y_pred_prob, y_tst)
        #print(fdr)
        FDRs.append(fdr)
        new = True
        if verbose:
            print('CV {} processed, FDR={:.2f}%'.format(i+1, fdr))
        
    return np.array(FDRs), np.array(FDRs).mean()

# one bug: total param combinations/njobs has to be >= 2
# set njobs smaller when you don't have that many param combinations

### Summary (FDR)

In [28]:
def summary(model):
    '''
    Summarize FDR on training/testing/oot data after model is fitted
    '''
    y_train_pred_prob = model.predict_proba(X_train)[:,1]
    y_test_pred_prob = model.predict_proba(X_test)[:,1]
    y_oot_pred_prob = model.predict_proba(X_oot)[:,1]
    print('Train FDR at 3%: {:.2f}%'.format(fdr_score(y_train_pred_prob,y_train)))
    print('Test FDR at 3%: {:.2f}%'.format(fdr_score(y_test_pred_prob,y_test)))
    print('OOT FDR at 3%: {:.2f}%'.format(fdr_score(y_oot_pred_prob,y_oot)))
    
#  print(pd.DataFrame([X_train.columns, model.feature_importances_]).T.sort_values(1,ascending=False).rename({0:"feature",1:"importance"},axis=1))

## Modeling

### Logistic Regression

In [37]:
y_oot = oot['fraud_label']
x_tt = tt.iloc[:,:30]
y = tt['fraud_label']

X_tr, X_ts, Y_trn, Y_tst = train_test_split(x_tt, y, test_size = .3)
FDR=pd.DataFrame()

In [38]:
for i in [10,15,20,25,30]:
    X_trn=X_tr.iloc[:,0:i]
    X_tst=X_ts.iloc[:,0:i]

    model = LogisticRegression(penalty='l2', C=.01, n_jobs=-1)
   
    X_oot = oot.iloc[:,0:i].copy()
    X_trn_eval = X_trn.copy()
    X_tst_eval = X_tst.copy()
    X_oot_eval = X_oot.copy()
    Y_oot = oot.iloc[:,31]
    y_oot=Y_oot
   
    X_trn_array = X_trn.values
    X_tst_array = X_tst.values
    X_oot_array = X_oot.values
    Y_trn_array = Y_trn.values
    Y_tst_array = Y_tst.values
    Y_oot_array = Y_oot.values

    model.fit(X_trn, Y_trn.values.ravel())  

    predictions = model.predict_proba(X_trn)[:,1]
    X_trn_eval['predicted']=predictions
    X_trn_eval['Fraud'] = Y_trn
    topRows = int(round(X_trn_eval.shape[0]*0.03))
    temp = X_trn_eval.sort_values('predicted',ascending=False).head(topRows)
    needed = temp.loc[:,'Fraud']
    FDR.loc[i, 'trn'] = sum(needed)/sum(X_trn_eval.loc[:,'Fraud'])

    predictions = model.predict_proba(X_tst)[:,1]
    X_tst_eval['predicted']=predictions
    X_tst_eval['Fraud'] = Y_tst
    topRows = int(round(X_tst_eval.shape[0]*0.03))
    temp = X_tst_eval.sort_values('predicted',ascending=False).head(topRows)
    needed = temp.loc[:,'Fraud']
    FDR.loc[i, 'tst'] = sum(needed)/sum(X_tst_eval.loc[:,'Fraud'])

    predictions = model.predict_proba(X_oot)[:,1]
    X_oot_eval['predicted']=predictions
    X_oot_eval['Fraud'] = y_oot
    topRows = int(round(X_oot_eval.shape[0]*0.03))
    temp = X_oot_eval.sort_values('predicted',ascending=False).head(topRows)
    needed = temp.loc[:,'Fraud']
    FDR.loc[i, 'oot'] = sum(needed)/sum(X_oot_eval.loc[:,'Fraud'])
    print(i, FDR.loc[i, 'trn'],FDR.loc[i, 'tst'],FDR.loc[i, 'oot'])

10 0.35310980929826746 0.3540282991625758 0.31391450125733444
15 0.3470023681914496 0.34940802772162866 0.31181894383906117
20 0.5227471020815156 0.5272884781980941 0.4979044425817267
25 0.5158918110432507 0.5194917701414958 0.49161777032690696
30 0.5181353608375919 0.5215131388969102 0.49245599329421624


In [39]:
FDR

,trn,tst,oot
10,0.353110,0.354028,0.313915
15,0.347002,0.349408,0.311819
20,0.522747,0.527288,0.497904
25,0.515892,0.519492,0.491618
30,0.518135,0.521513,0.492456


### Random Forest

In [40]:
# You can try to use other parameters or change values.
model = RandomForestClassifier(n_estimators=100, max_depth=None, max_features=5, min_samples_leaf=1,
                              min_samples_split=10, criterion="gini")

# cross validation on TRAIN+TEST data
fdr_tst, tst_avg = cross_val_fdr(model, X, y, cv=10, verbose=True)
print('tst_avg:',tst_avg)

# calculate avg fdr score(train/test/oot) for the initial train_test_split data 
score_summary, avg_scores = fit_avg_fdr(model, k=5)

Start cross_val_fdr...
CV 1 processed, FDR=55.45%
CV 2 processed, FDR=54.89%
CV 3 processed, FDR=55.77%
CV 4 processed, FDR=55.92%
CV 5 processed, FDR=52.66%
CV 6 processed, FDR=55.29%
CV 7 processed, FDR=58.33%
CV 8 processed, FDR=53.39%
CV 9 processed, FDR=57.36%
CV 10 processed, FDR=57.74%
tst_avg: 55.68009704434708


In [41]:
avg_scores

array([56.27626233, 56.60745385, 45.69991618,  0.95550045])

### XGBoost

In [30]:
result_arrays_list=[]
eta_list=[0.05,0.1,0.2,0.3]
max_depth_list=[2,3,4,5]
min_child_weight_list=[1,2]
turn=1
for eta in eta_list:
  print(f'round = {turn}')
  print(f'current eta = {eta}')
  for max_depth in max_depth_list:
    print(f'current max_depth = {max_depth}')
    for min_child_weight in min_child_weight_list:
      print(f'current min_child_weight = {min_child_weight}')
    # change n_jobs to a smaller number if your computer does not have so many CPUs
      model = XGBClassifier(n_jobs=3,
                            # use_label_encoder and eval_metric are specified to stop warnings from showing up
                            # these two parameters are set to default values
                            use_label_encoder=False,
                            eval_metric='logloss',
                            max_depth=max_depth,
                            eta=eta,
                            min_child_weight=min_child_weight)
      fdr_tst, tst_avg = cross_val_fdr(model, X, y, cv=10, verbose=False)
      print('tst_avg:',tst_avg)
      score_summary, avg_scores = fit_avg_fdr(model, k=5)
      print(f'round 1 result: {[turn,eta,max_depth,min_child_weight,tst_avg,avg_scores[0],avg_scores[1],avg_scores[2],avg_scores[3]]}')
      result_arrays_list.append([turn,eta,max_depth,min_child_weight,tst_avg,avg_scores[0],avg_scores[1],avg_scores[2],avg_scores[3]])
  turn+=10

round = 1
current eta = 0.05
current max_depth = 2
current min_child_weight = 1
Start cross_val_fdr...
tst_avg: 55.555570148395795
round 1 result: [1, 0.05, 2, 1, 55.555570148395795, 55.76784677887406, 56.18948101706722, 53.126571668063704, 0.9538310557695351]
current min_child_weight = 2
Start cross_val_fdr...
tst_avg: 55.75229279612987
round 1 result: [1, 0.05, 2, 2, 55.75229279612987, 55.59837492745212, 55.95262974573319, 52.79128248113999, 0.9538015089959373]
current max_depth = 3
current min_child_weight = 1
Start cross_val_fdr...
tst_avg: 56.1820755652584
round 1 result: [1, 0.05, 3, 1, 56.1820755652584, 56.02321532211259, 56.39846743295019, 53.386420787929595, 0.954502189626972]
current min_child_weight = 2
Start cross_val_fdr...
tst_avg: 55.980341483358245
round 1 result: [1, 0.05, 3, 2, 55.980341483358245, 56.04643064422519, 56.44723092998955, 53.386420787929595, 0.9541096396348863]
current max_depth = 4
current min_child_weight = 1
Start cross_val_fdr...
tst_avg: 55.926864625

In [31]:
output=pd.DataFrame(result_arrays_list,columns=['turn','eta','max_depth','min_child_weight','tst_avg','fdr_trainList', 'fdr_testList', 'fdr_ootList', 'accuracyList'])
output

,turn,eta,max_depth,min_child_weight,tst_avg,fdr_trainList,fdr_testList,fdr_ootList,accuracyList
0,1,0.05,2,1,55.555570,55.767847,56.189481,53.126572,0.953831
1,1,0.05,2,2,55.752293,55.598375,55.952630,52.791282,0.953802
2,1,0.05,3,1,56.182076,56.023215,56.398467,53.386421,0.954502
3,1,0.05,3,2,55.980341,56.046431,56.447231,53.386421,0.954110
4,1,0.05,4,1,55.926865,56.032501,56.440265,53.378039,0.955228
5,1,0.05,4,2,56.275420,56.013929,56.412400,53.352892,0.955083
6,1,0.05,5,1,55.874969,56.083575,56.426332,53.478625,0.955064
7,1,0.05,5,2,56.121498,56.090540,56.398467,53.445096,0.955068
8,11,0.10,2,1,55.846022,55.976785,56.335772,53.411567,0.953932
9,11,0.10,2,2,56.075301,55.979106,56.314873,53.419950,0.953877


### LightGBM

In [42]:
from lightgbm import LGBMClassifier
model = LGBMClassifier(n_jobs=1,
                       verbose=-1,
                       num_leaves=15,
                       n_estimators=700,
                       max_depth=5,
                       min_child_samples=200,
                       subsample=0.6,
                       colsample_bytree=0.8,
                       learning_rate=0.05)

# cross validation on TRAIN+TEST data
fdr_tst, tst_avg = cross_val_fdr(model, X, y, cv=10, verbose=True)
print('tst_avg:',tst_avg)

# calculate avg fdr score(train/test/oot) for the initial train_test_split data 
score_summary, avg_scores = fit_avg_fdr(model, k=5)

Start cross_val_fdr...
CV 1 processed, FDR=57.45%
CV 2 processed, FDR=56.06%
CV 3 processed, FDR=55.77%
CV 4 processed, FDR=57.16%
CV 5 processed, FDR=58.08%
CV 6 processed, FDR=54.36%
CV 7 processed, FDR=58.05%
CV 8 processed, FDR=56.89%
CV 9 processed, FDR=55.73%
CV 10 processed, FDR=56.26%
tst_avg: 56.58070045680364


In [43]:
avg_scores

array([56.18107951, 56.5308255 , 44.53478625,  0.95544135])

### Neural Network

In [44]:
model = MLPClassifier(hidden_layer_sizes=(50,), activation='relu', solver='adam', max_iter=200, random_state=1)
score_summary, avg_scores = fit_avg_fdr(model, k=5)
avg_scores

array([56.11375508, 56.47509579, 42.7409891 ,  0.95534005])

## Get Key Statistics in Top 20 Bins

In [32]:
X_train_undersampled, y_train_undersampled = trn_resample(X_train, y_train)

# This model achieved the highest FDR on test data, and was selected as the final model.
model = LGBMClassifier(n_jobs=1,
                       verbose=-1,
                       num_leaves=20,
                       n_estimators=600,
                       max_depth=5,
                       min_child_samples=200,
                       subsample=0.6,
                       colsample_bytree=0.8,
                       learning_rate=0.05)
model.fit(X_train_undersampled, y_train_undersampled)

y_train_pred_prob = model.predict_proba(X_train)[:,1]
y_test_pred_prob = model.predict_proba(X_test)[:,1]
y_oot_pred_prob = model.predict_proba(X_oot)[:,1]

In [33]:
def get_results(y, y_pred):
    df = pd.DataFrame({'actual': y.to_list(), 'pred': y_pred})
    df = df.sort_values(by='pred', ascending=False)
    bin_length = int(np.ceil(y.shape[0] / 100))
    print(f'# Records: {y.shape[0]}')
    print(f'# Bads: {y.sum()}')
    print(f'Bin length: {bin_length}')
    print('')
    print('Bads count in each bin')
    for i in range(20):
        print(df.iloc[i*bin_length:(i+1)*bin_length, :].actual.sum())

In [34]:
get_results(y_train, y_train_pred_prob)

# Records: 596247
# Bads: 8615
Bin length: 5963

Bads count in each bin
4573
178
90
55
57
52
61
43
47
48
41
37
38
38
36
34
39
37
34
40


In [35]:
get_results(y_test, y_test_pred_prob)

# Records: 198749
# Bads: 2871
Bin length: 1988

Bads count in each bin
1542
51
31
16
15
13
22
15
13
16
11
11
18
13
11
12
10
20
12
8


In [36]:
get_results(y_oot, y_oot_pred_prob)

# Records: 166493
# Bads: 2386
Bin length: 1665

Bads count in each bin
1218
38
22
16
15
20
9
17
20
7
16
9
12
13
12
15
20
13
14
10
